# 使用データセット
- [train set](https://pjreddie.com/media/files/mnist_train.csv)
- [test set](https://pjreddie.com/media/files/mnist_test.csv)


In [ ]:
import numpy
# scipy.specialのインポート。シグモイド関数 expit() 利用のため
import scipy.special
import matplotlib.pyplot
%matplotlib inline

class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        """
        ニューラルネットワークの初期化: 入力層、隠れ層および入力層の各層のノード数を設定する
        """
        # 入力層、隠れ層、出力層のノード数の設定
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # リンクの重み行列 wih と who
        # 行列内の重み w_i_j, ノード i から次のそうのノードj へのリンクの重み
        # w11 w21
        # w12 w22 など
        self.wih = numpy.random.normal(
            0.0,
            pow(self.hnodes, -0.5),
            (self.hnodes, self.inodes))
        self.who = numpy.random.normal(
            0.0,
            pow(self.onodes, -0.5),
            (self.onodes, self.hnodes))

        # 学習率の設定
        self.lr = learningrate

        # 活性化関数はシグモイド関数
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    def train(self, inputs_list, targets_list):
        """
        ニューラルネットワークの学習: 与えられた訓練データから重みを調整する
        """
        # 入力リストを行列に変換
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T

        # 隠れ層に入ってくる信号の計算
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 隠れ層で結合された信号を活性化関数により出力
        hidden_outputs = self.activation_function(hidden_inputs)

        # 出力層に入ってくる信号の計算
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 出力層で結合された信号を活性化関数により出力
        final_outputs = self.activation_function(final_inputs)

        # 出力層の誤差 = (目標出力 - 最終出力)
        output_errors = targets - final_outputs
        # 隠れ層の誤差は出力層の誤差をリンクの重みの割合で分配
        hidden_errors = numpy.dot(self.who.T, output_errors)

        # 隠れ層と出力層の間のリンクの重みを更新
        self.who += self.lr * numpy.dot(
                (output_errors * final_outputs * (1.0 - final_outputs)),
                numpy.transpose(hidden_outputs))

        # 入力層と隠れ層の間のリンクの重みを更新
        self.wih += self.lr * numpy.dot(
                (hidden_errors * hidden_outputs * (1.0 - hidden_outputs)),
                numpy.transpose(inputs))
        pass

    def query(self, inputs_list):
        """
        ニューラルネットワークへの照会: 与えられた入力に対する出力層からの答えを返す
        """
        # 入力リストを行列に変換
        inputs = numpy.array(inputs_list, ndmin=2).T

        # 隠れ層に入ってくる信号の計算
        hidden_inputs = numpy.dot(self.wih, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        # 出力層に入ってくる信号の計算
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 出力層で結合された信号を活性化関数により出力
        final_outputs = self.activation_function(final_inputs)

        return final_outputs


In [ ]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

# 学習率 = 0.3
learning_rate = 0.3
# ニューラルネットワークのインスタンスの生成
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
# MNIST 訓練データのCSVファイルを読み込んでリストにする
training_data_file = open("mnist_dataset/mnist_train_100.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# ニューラルネットワークの学習

# 訓練データの全データに対して実行
for record in training_data_list:
  # データをコンマ ',' でsplit
  all_values = record.split(',')
  # 入力値のスケーリングとシフト
  inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
  # 目標配列の生成（ラベルの位置が0.99 残りは0.01
  targets = numpy.zeros(output_nodes) + 0.01
  # all_valuesp[0] はこのデータのラベル
  targets[int(all_values[0])] = 0.99
  n.train(inputs, targets)
  pass

In [ ]:
# MNIST テストデータのCSVファイルを読み込んでリストにする
test_data_file = open("mnist_dataset/mnist_test.csv", mode='r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [ ]:
# テストデータセットの最初のデータを取得
all_values = test_data_list[0].split(',')
# ラベルを出力
print(all_values[0])

In [ ]:
image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
matplotlib.pyplot.imshow(image_array, cmap='Greys', interpolation='None')

In [ ]:
n.query(numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

In [ ]:
scaled_input = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
print(scaled_input)

In [ ]:
onodes = 10
targets = numpy.zeros(onodes) + 0.01
targets[int(all_values[0])] = 0.99

In [ ]:
print(targets)

In [ ]:
# ニューラルネットワークのテスト

# scorecard は判定のリスト、最初は空
scorecard = []

# テストデータの全てのデータに対して実行
for record in test_data_list:
  # データをコンマ ',' でsplit
  all_values = record.split(',')
  # 正解は配列の1番目
  correct_label = int(all_values[0])
  print(correct_label, "correct label")
  # 入力値のスケーリングとシフト
  inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
  # ネットワークへの照会
  outputs = n.query(inputs)
  label = numpy.argmax(outputs)
  print(label, "network's answer")
  # 正解(1), 間違い(0) をリストに追加
  if (label == correct_label):
    # 正解なら1 を追加
    scorecard.append(1)
  else:
    scorecard.append(0)
    pass
  pass


In [ ]:
print(scorecard)

In [ ]:
# 評価値（正解の割合）の計算
scorecard_array = numpy.asarray(scorecard)
print("performance = ", scorecard_array.sum() / scorecard_array.size)